## Loading data and import libraries

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as mpl
import sqlite3
%matplotlib inline
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth',5000)

In [2]:
park_codes = pd.read_csv('park_codes.csv')
person_codes = pd.read_csv('person_codes.csv')
team_codes = pd.read_csv('team_codes.csv')

In [3]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [4]:
datatypes = {'day_night':object, 'completion':object, 'forefeit':object,
            'protest':object, 'v_line_score':object, 'h_line_score':object,
            '2b_umpire_id':object, '2b_umpire_name':object, '3b_umpire_id':object,
            '3b_umpire_name':object, 'lf_umpire_id':object, 'lf_umpire_name':object,
            'rf_umpire_id':object, 'rf_umpire_name':object, 'winning_pitcher_id':object,
            'winning_pitcher_name':object, 'losing_pitcher_id':object, 'losing_pitcher_name':object,
            'saving_pitcher_id':object, 'saving_pitcher_name':object, 'winning_rbi_batter_id':object,
            'winning_rbi_batter_id_name':object, 'v_player_1_id':object, 'v_player_1_name':object,
            'v_player_2_id':object, 'v_player_2_name':object, 'v_player_3_id':object,
            'v_player_3_name':object, 'v_player_4_id':object, 'v_player_4_name':object,
            'v_player_5_id':object, 'v_player_5_name':object, 'v_player_6_id':object,
            'v_player_6_name': object, 'v_player_7_id':object, 'v_player_7_name':object,
            'v_player_8_id':object, 'v_player_8_name':object, 'v_player_9_id':object,
            'v_player_9_name':object, 'h_player_1_id':object, 'h_player_1_name':object,
            'h_player_2_id':object, 'h_player_2_name':object, 'h_player_3_id':object,
            'h_player_3_name':object, 'h_player_4_id':object, 'h_player_4_name':object,
            'h_player_5_id':object, 'h_player_5_name':object, 'h_player_6_id':object,
            'h_player_6_name':object, 'h_player_7_id':object, 'h_player_7_name':object,
            'h_player_8_id':object, 'h_player_8_name':object, 'h_player_9_id':object,
            'h_player_9_name':object, 'acquisition_info':object}
game_logs = pd.read_csv('game_log.csv', dtype=datatypes)

## EDA - Null Analysis

In [5]:
game_logs_nulls = (round(game_logs.isnull().sum().astype(float) / float(game_logs.shape[0]) * 100,2).sort_values(ascending=False)[:15].astype(str)) + '%'
print(game_logs_nulls)

rf_umpire_name                100.0%
rf_umpire_id                  100.0%
completion                    99.94%
protest                       99.88%
forefeit                      99.87%
lf_umpire_name                99.82%
lf_umpire_id                  99.82%
additional_info               98.88%
saving_pitcher_name           83.44%
saving_pitcher_id             83.44%
2b_umpire_name                73.74%
2b_umpire_id                  73.74%
winning_rbi_batter_id_name    55.99%
winning_rbi_batter_id         55.99%
v_intentional_walks           51.81%
dtype: object


In [6]:
park_codes_nulls = (round(park_codes.isnull().sum().astype(float) / float(park_codes.shape[0]) * 100,2).sort_values(ascending=False)[:15].astype(str)) + '%'
print(park_codes_nulls)

aka        76.98%
notes      49.21%
league     26.19%
end         11.9%
start        0.0%
state        0.0%
city         0.0%
name         0.0%
park_id      0.0%
dtype: object


In [7]:
person_codes_nulls = (round(person_codes.isnull().sum().astype(float) / float(person_codes.shape[0]) * 100,2).sort_values(ascending=False)[:15].astype(str)) + '%'
print(person_codes_nulls)

mgr_debut       96.58%
ump_debut       92.46%
coach_debut     92.36%
player_debut      6.4%
first             0.3%
last              0.0%
id                0.0%
dtype: object


In [8]:
team_codes_nulls = (round(team_codes.isnull().sum().astype(float) / float(team_codes.shape[0]) * 100,2).sort_values(ascending=False)[:15].astype(str)) + '%'
print(team_codes_nulls)

league       17.33%
seq            0.0%
franch_id      0.0%
nickname       0.0%
city           0.0%
end            0.0%
start          0.0%
team_id        0.0%
dtype: object


## EDA - Table Dimensions

In [9]:
game_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112646 entries, 0 to 112645
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 138.4+ MB


In [10]:
print("File dimensions")
print('game_logs: ', game_logs.shape)
print('park_codes: ', park_codes.shape)
print('person_codes: ', person_codes.shape)
print('team_codes: ', team_codes.shape)

File dimensions
game_logs:  (112646, 161)
park_codes:  (252, 9)
person_codes:  (20494, 7)
team_codes:  (150, 8)


## EDA - Table Head Records

In [11]:
game_logs.head()

date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id  attendance  length_minutes v_line_score  \
0      NaN     NaN   FOR01       200.0           120.0    000000000   
1      NaN     NaN   WAS01      5000.0           145.0    107000435   
2      NaN     NaN   RCK01      1000.0           140.0    610020003   
3      NaN     NaN   CHI01      5000.0           150.0    101403111   
4      NaN     NaN   TRO01      3250.0           145.0    000002232   

  h_line_score  v_at_bats  v_hits  v_doubles  v_triples  v_homeruns  v_rbi  \
0    010010000       30.0     4.0        1.0        0.0         0.0    0.0   
1    640113030       41.0    13.0        1.0        2.0         0.0   13.0   
2    010020100       49.0    11.0        1.0        1.0         0.0    8.0   
3    077000000       46.0    15.0        2.0        1.0         2.0   10.0   
4    101003000       46.0    17.0        4.0        1.0         0.0    6.0   

   v_sacrifice_hits  v_sacrifice_flies  v_hit_by_pitch  v_walks  \
0               0.0                0.0             0.0      1.0   
1               0.0                0.0             0.0     18.0   
2               0.0                0.0             0.0      0.0   
3               0.0                0.0             0.0      0.0   
4               0.0                0.0             0.0      2.0   

   v_intentional_walks  v_strikeouts  v_stolen_bases  v_caught_stealing  \
0                  NaN           6.0             1.0                NaN   
1                  NaN           5.0             3.0                NaN   
2                  NaN           1.0             0.0                NaN   
3                  NaN           1.0             0.0                NaN   
4                  NaN           0.0             1.0                NaN   

   v_grounded_into_double  v_first_catcher_interference  v_left_on_base  \
0                    -1.0                           NaN             4.0   
1                    -1.0                           NaN            12.0   
2                    -1.0                           NaN            10.0   
3                    -1.0                           NaN             7.0   
4                    -1.0                           NaN            12.0   

   v_pitchers_used  v_individual_earned_runs  v_team_earned_runs  \
0              1.0                       1.0                 1.0   
1              1.0                       6.0                 6.0   
2              1.0                       0.0                 0.0   
3              1.0                       6.0                 6.0   
4              1.0                       2.0                 2.0   

   v_wild_pitches  v_balks  v_putouts  v_assists  v_errors  v_passed_balls  \
0             0.0      0.0       27.0        9.0       0.0             3.0   
1             1.0      0.0       27.0       13.0      10.0             1.0   
2             2.0      0.0       27.0       12.0       8.0             5.0   
3             0.0      0.0       27.0       15.0      11.0             6.0   
4             0.0      0.0       

In [12]:
park_codes.head()

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             NaN  
3  10/03/1993     AL                                             NaN  
4         NaN     AL                                             NaN

In [13]:
person_codes.head()

id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN

In [14]:
team_codes.head()

team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1

In [15]:
team_codes['end'].value_counts()[:10]

0       30
1884    19
1875    13
1890    12
1891     9
1915     8
1872     6
1889     5
1899     4
1877     3
Name: end, dtype: int64

## Observations

### Game Logs:

* File has all the events and characteristics of a specific game inclunding game stats, player line-up and additional information.
* Columns with high percentage of null values should probably be deprecated:
    * rf_umpire_name
    * rf_umpire_id
    * completion
    * forefeit
    * protest
    * lf_umpire_name
    * lf_umpire_id
    * additional_info
* Foreign Keys:
    * v_name
    * h_name
    * park_id
    * player_X_id
  
### Park Codes:

* This is a look-up table with park_id as PK and FK.
* aka column has 3/4 of values as nulls, we will not consider it as part of the analysis. 

### Person Codes:

* This is a look-up table with id as PK and FK.
* This table has very high null percentage for columns mgr_debut, ump_debut and coach_debut.

### Team Codes:

* Look-up table with team_id as PK and FK.
* Overall low null percentage on all tables with column league with most nulls. 

## Set Up Database

In [16]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = """
    SELECT 
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table", "view");
    """
    return run_query(q)

In [17]:
tables = {
    "game_logs": game_logs,
    "person_codes": person_codes,
    "team_codes": team_codes,
    "park_codes": park_codes
}

with sqlite3.connect("mlb.db") as conn:
    for name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {}".format(name))
        data.to_sql(name, conn, index=False)

In [18]:
show_tables()

name   type
0     game_logs  table
1  person_codes  table
2    team_codes  table
3    park_codes  table

In [19]:
## Add an primary key to main table "game_logs"
add_column = """ALTER TABLE game_logs
    ADD COLUMN game_id TEXT;

"""
add_values = """UPDATE game_logs
    SET game_id = (h_name || date || number_of_game)
    WHERE game_id IS NULL;

"""
run_command(add_column)
run_command(add_values)

In [20]:
run_query('select game_id from game_logs limit 10;')

game_id
0  FW1187105040
1  WS3187105050
2  RC1187105060
3  CH1187105080
4  TRO187105090
5  CL1187105110
6  CL1187105130
7  FW1187105130
8  FW1187105150
9  BS1187105160

## Observations

* On game_logs table, the pitcher and batter columns can be normalized with only the id and referencing to the player_codes table.
* Probably on the player_codes file we can obtain player debuts from the game_logs and omit the debut table due to redundancy. 

## Schema Model Design

In order to normalize de dataset, a new schema model was designed to remove repetetive columns and redundant data. 

![Alt text](https://mezzog.bl.files.1drv.com/y4m2mYVVxxoTqL2GJhr1suUxnaPG03DtNDqk-WQMqeSS-vQ4TWR9NUzWAjS1Q8ZYBbWSrpGYAigwuIZ-Fne0BMG24DzWD6xeQ4HqP6lspKbumT9VvfULeL7_-ylAfeOMCxXrXxkUvSlHUNmckNUlQxm5uFWRyGDGM8rOqF8-wG6bcjbSPUQbjx4VbsiofmL74NwRp8rfztTp7FnLcEQgJz4uQ?width=1021&height=808&cropmode=none)

## Create person table

Create person table according to new schema model.
* 'debut' columns have been omitted, this data can be extracted from other tables. 
* Since the game_logs has no data on coaches besides the name, the data was removed from the dataset. 

In [21]:
# table created for person
person_table = """CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
)
"""

#data inserted from the person_codes table
person_table_query = """INSERT INTO person
    SELECT 
        id,
        first,
        last
    FROM person_codes;
"""
run_command(person_table)
run_command(person_table_query)

In [22]:
run_query("SELECT * FROM person LIMIT 5;") #check new finished table

person_id first_name last_name
0  aardd001      David   Aardsma
1  aaroh101       Hank     Aaron
2  aarot101     Tommie     Aaron
3  aased001        Don      Aase
4  abada001       Andy      Abad

## Create park table

Created park table with columns and primary key as shown in the schema diagram.

* Inserted the data from the park_codes table.

In [23]:
park_table = """CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
);
"""
park_table_values = """INSERT INTO park
    SELECT
        park_id,
        name,
        aka,
        city,
        state,
        notes
    FROM park_codes;
"""
run_command(park_table)
run_command(park_table_values)

In [24]:
run_query("SELECT * FROM park LIMIT 5;")

park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                   nickname       city state  \
0                                      None     Albany    NY   
1                                      None    Altoona    PA   
2             Edison Field; Anaheim Stadium    Anaheim    CA   
3                                      None  Arlington    TX   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX   

                                            notes  
0  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1                                            None  
2                                            None  
3                                            None  
4                                            None

## Create league table

Created league table with columns and primary key as shown in the schema diagram.

* Inserted manually data from research of the names of the leagues 

In [25]:
game_logs['v_league'].value_counts().index

Index(['NL', 'AL', 'AA', 'FL', 'PL', 'UA'], dtype='object')

In [26]:
league_table = """CREATE TABLE IF NOT EXISTS league(
    league_id TEXT PRIMARY KEY,
    name TEXT
);
"""

league_table_values = """INSERT INTO league
    VALUES
        ('NL','National League'),
        ('AL', 'American League'),
        ('AA', 'American Association'),
        ('FL', 'Federal League'),
        ('PL', 'Players League'),
        ('UA', 'Union Association');
"""
run_command(league_table)
run_command(league_table_values)

In [27]:
run_query("SELECT * FROM league;")

league_id                  name
0        NL       National League
1        AL       American League
2        AA  American Association
3        FL        Federal League
4        PL        Players League
5        UA     Union Association

## Create appearance_type table

Created table with columns and primary key as shown in the schema diagram.

* This time we will import data from a file 'appearance_type.csv'

In [28]:
c1 = "DROP TABLE IF EXISTS appearance_type;"
run_command(c1)

appearance_type_table = """CREATE TABLE appearance_type(
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
);
"""
run_command(appearance_type_table)

appearance_type = pd.read_csv('appearance_type.csv')
with sqlite3.connect('mlb.db') as conn:
    appearance_type.to_sql('appearance_type', con=conn, index=False, if_exists='append')

In [29]:
run_query("SELECT * FROM appearance_type;")

appearance_type_id                name category
0                  O1            Batter 1  offense
1                  O2            Batter 2  offense
2                  O3            Batter 3  offense
3                  O4            Batter 4  offense
4                  O5            Batter 5  offense
5                  O6            Batter 6  offense
6                  O7            Batter 7  offense
7                  O8            Batter 8  offense
8                  O9            Batter 9  offense
9                  D1             Pitcher  defense
10                 D2             Catcher  defense
11                 D3            1st Base  defense
12                 D4            2nd Base  defense
13                 D5            3rd Base  defense
14                 D6           Shortstop  defense
15                 D7          Left Field  defense
16                 D8        Center Field  defense
17                 D9         Right Field  defense
18                D10    Unknown Position  defense
19                UHP          Home Plate   umpire
20                U1B          First Base   umpire
21                U2B         Second Base   umpire
22                U3B          Third Base   umpire
23                ULF          Left Field   umpire
24                URF         Right Field   umpire
25                 MM             Manager  manager
26                AWP     Winning Pitcher    award
27                ALP      Losing Pitcher    award
28                ASP      Saving Pitcher    award
29                AWB  Winning RBI Batter    award
30                PSP    Starting Pitcher  pitcher

## Create team table

Created team table with columns, primary key, and foreign key as shown in the schema diagram.

* Inserted the data from the team_codes table.

In [30]:
## Checked for any duplicates
team_codes['team_id'].value_counts().sort_values(ascending=False)[:5]

MIL    2
WS1    1
IN2    1
CHU    1
SLN    1
Name: team_id, dtype: int64

In [31]:
team_codes[team_codes['team_id'] == 'MIL']

team_id league  start   end       city nickname franch_id  seq
112     MIL     AL   1970  1997  Milwaukee  Brewers       SE1    2
113     MIL     NL   1998     0  Milwaukee  Brewers       SE1    3

In [32]:
## Removed duplicate row and updated start column with oldest start date
## Milwaukee Brewers apparently changed league and caused duplicate records on dataset
update_query = """UPDATE team_codes
    SET start = 1970
    WHERE team_id = 'MIL' AND league = 'NL';
"""
remove_query = """DELETE FROM team_codes
    WHERE team_id = 'MIL' AND league = 'AL';
"""
run_command(update_query)
run_command(remove_query)

In [33]:
team_table = """CREATE TABLE IF NOT EXISTS team(
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
"""
team_table_query = """INSERT INTO team
    SELECT
        team_id,
        league,
        city,
        nickname,
        franch_id
    FROM team_codes;
"""

run_command(team_table)
run_command(team_table_query)

In [34]:
run_query("SELECT * FROM team LIMIT 5;")

team_id league_id       city         nickname franch_id
0     ALT        UA    Altoona  Mountain Cities       ALT
1     ARI        NL    Arizona     Diamondbacks       ARI
2     BFN        NL    Buffalo           Bisons       BFN
3     BFP        PL    Buffalo           Bisons       BFP
4     BL1      None  Baltimore         Canaries       BL1

## Create game table

Created table with columns, primary key, and foreign key as shown in the schema diagram.

* Inserted data from the game_log table.

In [35]:
game_table = """CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date INT,
    number_of_game INT,
    park_id TEXT,
    length_outs INT,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INT,
    length_minutes INT,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
"""
game_table_values = """INSERT INTO game
    SELECT
        game_id,
        date,
        number_of_game,
        park_id,
        length_outs,
        day_night,
        completion,
        forefeit,
        protest,
        attendance,
        length_minutes,
        additional_info,
        acquisition_info
    FROM game_logs;
"""
run_command(game_table)
run_command(game_table_values)

In [36]:
run_query("SELECT * FROM game LIMIT 5;")

game_id      date  number_of_game park_id  length_outs day completion  \
0  FW1187105040  18710504               0   FOR01           54   D       None   
1  WS3187105050  18710505               0   WAS01           54   D       None   
2  RC1187105060  18710506               0   RCK01           54   D       None   
3  CH1187105080  18710508               0   CHI01           54   D       None   
4  TRO187105090  18710509               0   TRO01           54   D       None   

  forefeit protest  attendance  length_minutes additional_info  \
0     None    None         200             120            None   
1     None    None        5000             145            HTBF   
2     None    None        1000             140            None   
3     None    None        5000             150            None   
4     None    None        3250             145            HTBF   

  acquisition_info  
0                Y  
1                Y  
2                Y  
3                Y  
4                Y

## Create team_appearance table

Create the team_appearance table with columns, primary key, and foreign keys as shown in the schema diagram.

* Inserted the data from the game_log table, using a UNION clause to combine the data from the column sets for the home and away teams.

In [37]:
query = """SELECT sql FROM sqlite_master
    where name = 'game_logs'
    and type = 'table';

"""

sql = run_query(query)
sql.head()

sql
0  CREATE TABLE "game_logs" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_play

In [38]:
split = (sql['sql']
         .str.replace('"', '')
         .str.replace('[A-Z]','')
         .str.replace(' ','')
         .str.split('\n'))
home = []
for s in split[0]:
    encontrado = re.search('h_', s)
    if encontrado:
        home.append(s)
home_query = ''.join(home[3:-31])
home_query

'h_score,length_outs,length_minutes,h_line_score,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,'

In [39]:
split = (sql['sql']
         .str.replace('"', '')
         .str.replace('[A-Z]','')
         .str.replace(' ','')
         .str.split('\n'))
away = []
for s in split[0]:
    encontrado = re.search('v_', s)
    if encontrado:
        away.append(s)
away_query = ''.join(away[3:-31])
away_query

'v_score,v_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,'

In [40]:
team_appearance_table = """CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
)
"""
team_appearance_table_values = """INSERT INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_logs
UNION
    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_logs;"""

run_command(team_appearance_table)
run_command(team_appearance_table_values)

In [41]:
run_query("SELECT * FROM team_appearance where hits not null LIMIT 5;")

team_id       game_id  home league_id  score     line_score  at_bats  hits  \
0     ATL  ATL196604120     1        NL      2  0000100000001       42     8   
1     ATL  ATL196604130     1        NL      0      000000000       30     4   
2     ATL  ATL196604220     1        NL      8      20301002x       38    13   
3     ATL  ATL196604230     1        NL      5      31000001x       29     9   
4     ATL  ATL196604241     1        NL      5      03011000x       34    12   

   doubles  triples  homeruns  rbi  sacrifice_hits  sacrifice_flies  \
0        0        0         2    2               2                0   
1        0        0         0    0               0                0   
2        3        0         0    6               1                0   
3        1        0         2    3               1                0   
4        2        0         0    5               0                0   

   hit_by_pitch  walks  intentional_walks  strikeouts  stolen_bases  \
0             0      8                  1           7             1   
1             0      0                  0           4             0   
2             0      6                  2           8             1   
3             1      6                  0           2             2   
4             0      6                  0           3             1   

   caught_stealing  grounded_into_double  first_catcher_interference  \
0                1                     2                           0   
1                0                     0                           0   
2                0                     1                           0   
3                0                     2                           0   
4                0                     1                           0   

   left_on_base  pitchers_used  individual_earned_runs  team_earned_runs  \
0            11              1                       3                 3   
1             3              4                       6                 6   
2            13              2                       2                 2   
3             8              2                       4                 4   
4            11              2                       2                 2   

   wild_pitches  balks  putouts  assists  errors  passed_balls  double_plays  \
0             2      0       39       15       1             0             2   
1             0      0       27       14       2             1             1   
2             0      0       27       12       1             0             1   
3             0      0       27       10       0             0             0   
4             0      0       27       14       0             0             1   

   triple_plays  
0             0  
1             0  
2             0  
3             0  
4             0

## Create person_appearance table

Created table with columns, primary key, and foreign keys as shown in the schema diagram.

* Inserted the data from the game_logs table, using UNION clauses to combined the data from the columns for managers, umpires, pitchers, and awards.
* Used a loop with string formatting to insert the data for offensive and defensive positions from the game_logs table.

In [42]:
run_query("SELECT * FROM appearance_type;")

appearance_type_id                name category
0                  O1            Batter 1  offense
1                  O2            Batter 2  offense
2                  O3            Batter 3  offense
3                  O4            Batter 4  offense
4                  O5            Batter 5  offense
5                  O6            Batter 6  offense
6                  O7            Batter 7  offense
7                  O8            Batter 8  offense
8                  O9            Batter 9  offense
9                  D1             Pitcher  defense
10                 D2             Catcher  defense
11                 D3            1st Base  defense
12                 D4            2nd Base  defense
13                 D5            3rd Base  defense
14                 D6           Shortstop  defense
15                 D7          Left Field  defense
16                 D8        Center Field  defense
17                 D9         Right Field  defense
18                D10    Unknown Position  defense
19                UHP          Home Plate   umpire
20                U1B          First Base   umpire
21                U2B         Second Base   umpire
22                U3B          Third Base   umpire
23                ULF          Left Field   umpire
24                URF         Right Field   umpire
25                 MM             Manager  manager
26                AWP     Winning Pitcher    award
27                ALP      Losing Pitcher    award
28                ASP      Saving Pitcher    award
29                AWB  Winning RBI Batter    award
30                PSP    Starting Pitcher  pitcher

In [43]:
c0 = "DROP TABLE IF EXISTS person_appearance"

run_command(c0)

person_appearance_table = """CREATE TABLE person_appearance(
    appearance_id INTEGER PRIMARY KEY,
    game_id TEXT,
    team_id TEXT,
    person_id TEXT,
    appearance_type_id,
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

person_appearance_table_values = """ INSERT INTO person_appearance(
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        'ULF'
    FROM game_logs
    WHERE lf_umpire_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        'URF'
    FROM game_logs
    WHERE rf_umpire_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        'UHP'
    FROM game_logs
    WHERE hp_umpire_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        'U1B'
    FROM game_logs
    WHERE [1b_umpire_id] IS NOT NULL
    
UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        'U2B'
    FROM game_logs
    WHERE [2b_umpire_id] IS NOT NULL   
    
UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        'U3B'
    FROM game_logs
    WHERE [3b_umpire_id] IS NOT NULL   
    
UNION

    SELECT 
        game_id,
        v_name,
        v_manager_id,
        'MM'
    FROM game_logs
    WHERE v_manager_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        'MM'
    FROM game_logs
    WHERE h_manager_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
        END,
        winning_pitcher_id,
        'AWP'
    FROM game_logs
    WHERE winning_pitcher_id IS NOT NULL
    
UNION

     SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
        END,
        losing_pitcher_id,
        'ALP'
    FROM game_logs
    WHERE losing_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
        END,
        saving_pitcher_id,
        'ASP'
    FROM game_logs
    WHERE saving_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
        END,
        winning_rbi_batter_id,
        'AWB'
    FROM game_logs
    WHERE winning_rbi_batter_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        'PSP'
    FROM game_logs
    WHERE h_starting_pitcher_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        'PSP'
    FROM game_logs
    WHERE v_starting_pitcher_id IS NOT NULL;
        
"""

run_command(person_appearance_table)
run_command(person_appearance_table_values)

In [44]:
person_appearance_values = """INSERT INTO person_appearance(
    game_id,
    team_id,
    person_id,
    appearance_type_id
)
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        'O{num}'
    FROM game_logs
    WHERE {hv}_player_{num}_id IS NOT NULL
    
UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        'D' || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_logs
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(person_appearance_values.format(**query_vars))

In [45]:
run_query("select * from person_appearance LIMIT 5;")

appearance_id       game_id team_id person_id appearance_type_id
0              1  ALT188404300    None  maplb901                UHP
1              2  ALT188404300     ALT  curte801                 MM
2              3  ALT188404300     ALT  murpj104                PSP
3              4  ALT188404300     SLU  hodnc101                PSP
4              5  ALT188404300     SLU  sullt101                 MM

In [46]:
#Dropped tables that were used to hold unnormalized data

drop_table_gl = "DROP TABLE game_logs;"
drop_table_pc = "DROP TABLE park_codes;"
drop_table_tc = "DROP TABLE team_codes;"
drop_table_perc = "DROP TABLE person_codes;"

run_command(drop_table_gl)
run_command(drop_table_pc)
run_command(drop_table_tc)
run_command(drop_table_perc)